- adicionar coluna valor de venda
- adicionar nota ENEM
- adicionar coords

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
import json
from utils.busca_ceps import cep_to_coords


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

In [ ]:
df = pd.read_csv("../dados/temporarios/02-md_edb_limpo.csv",
                 dtype={"CO_CEP": str})  # 37_178 linhas

df.info()

## Valor de venda

In [ ]:
with open("../dados/inputs/ticket_medio.json", "r", encoding="utf-8") as f:
    ticket_medio = json.load(f)


df["valor_venda"] = (
    df["QT_MAT_INF"] * ticket_medio["ei"]
    + df["QT_MAT_FUND_AI"] * ticket_medio["efai"]
    + df["QT_MAT_FUND_AF"] * ticket_medio["efaf"]
    + df["QT_MAT_MED"] * ticket_medio["em"]
    )


display(df.head())
print(df.shape)


## Nota Enem

In [ ]:
df_escolas_enem = pd.read_csv("../dados/temporarios/escolas_enem.csv")

df_escolas_enem.info()

In [ ]:
df = df.merge(df_escolas_enem, how="left", left_on="CO_ENTIDADE", right_on="cod_escola")
df = df.drop(columns=["cod_escola"])

display(df.head())
print(df.shape)

## Coordenadas

In [ ]:
df_final = await cep_to_coords(df, "CO_CEP")



## Salva

In [ ]:
display(df_final.head())
print(df_final.shape)

In [ ]:
df_final.info()

In [ ]:
df_final.to_csv("../dados/temporarios/10-md_edb_cols_finais.csv", index=False)